## Нейронный переводчик с английского на русский

В данном проекте мы будем реализовывать архитектуру seq2seq с механизмом attention. Более подробное описание всей работы архитектуры можно найти в текстовом файле. Здесь я буду писать кратко.

Сперва импортируем все необходимые библиотеки, напишем свою версию функции softmax и определим некоторые константы.

In [1]:
from __future__ import print_function, division
from builtins import range, input
import os, sys
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, \
  Bidirectional, RepeatVector, Concatenate, Activation, Dot, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
try:
    import keras.backend as K
    if len(K.tensorflow_backend._get_available_gpus()) > 0:
        from keras.layers import CuDNNLSTM as LSTM
        from keras.layers import CuDNNGRU as GRU
except:
    pass


# softmax должна вычисляться по временной оси
# т.к. дефолтная реализация ожидает, что временная ось находится в конце
# то нам нужно реализовать данную функцию самостоятельно
# ожидаемые размерности N x T x D (время по середине)
# note: the latest version of Keras allows you to pass in axis arg
def softmax_over_time(x):
    assert(K.ndim(x) > 2)
    e = K.exp(x - K.max(x, axis=1, keepdims=True))
    s = K.sum(e, axis=1, keepdims=True)
    return e / s



# константы
BATCH_SIZE = 64
EPOCHS = 30
LATENT_DIM = 400
LATENT_DIM_DECODER = 500 # надо проверить, что будет работать с разным количеством нейронов
NUM_SAMPLES = 20000
MAX_SEQUENCE_LENGTH = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

Using TensorFlow backend.


Загрузим датасет, который хранит в себе большое количество фраз на английском и их соответствующий перевод на русский. На его основе создадим 3 списка: в первом хранятся фразы на английском языке, во втором соответствующий перевод на русском плюс специальный токен сигнализирующий окончание предложения, а в третьем тоже соответствующие переводы только с добавлением токена о начале предложения. Последний нужен для того, чтобы обучить decoder через teacher forcing.

In [2]:
input_texts = [] 
target_texts = [] 
target_texts_inputs = [] 


t = 0
for line in open('rus.txt',encoding='utf-8'):
  # возьмем данные не из всего датасета
    t += 1
    if t > NUM_SAMPLES:
        break

    if '\t' not in line:
        continue

    input_text, translation, *rest = line.rstrip().split('\t')

    target_text = translation + ' <eos>'
    target_text_input = '<sos> ' + translation

    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_inputs.append(target_text_input)
print("Количество фраз:", len(input_texts))

Количество фраз: 20000


Следующим шагом будет создание токенизатора для каждого языка отдельно, определение словаря с парами слово-индекс и определение максимальной длины фразы как входного предложения, так и перевода.

In [3]:
tokenizer_inputs = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)

word2idx_inputs = tokenizer_inputs.word_index
print(f'Найдено {len(word2idx_inputs)} уникальных слов в Английском')

max_len_input = max(len(s) for s in input_sequences)

Найдено 2803 уникальных слов в Английском


In [4]:
# filters='' для того, чтобы токенизатор учел специальные токены
tokenizer_outputs = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
tokenizer_outputs.fit_on_texts(target_texts + target_texts_inputs) 
target_sequences = tokenizer_outputs.texts_to_sequences(target_texts)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_texts_inputs)

word2idx_outputs = tokenizer_outputs.word_index
print(f'Найдено {len(word2idx_outputs)} уникальных слов в Русском')

# сохраним число уникальных слов + 1 на будущее
# модель к каждому такому уникальному слову будет приписывать вероятность на основе которой она будет делать предсказания
num_words_output = len(word2idx_outputs) + 1

max_len_target = max(len(s) for s in target_sequences)

Найдено 9896 уникальных слов в Русском


Нужно сделать паддинг всех последовательностей до найденной максимальной длины для обоих языков.

Для Encoder'а добавляем нули вначале, чтобы ему не нужно было много запоминать. Для Decoder'а логичнее сделать паддинг в конце, так как перевод не может начаться с пустых символов.

In [5]:
encoder_inputs = pad_sequences(input_sequences, maxlen=max_len_input)
print("encoder_data.shape:", encoder_inputs.shape)
print("encoder_data[0]:", encoder_inputs[0])

decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=max_len_target, padding='post')
print("decoder_data[0]:", decoder_inputs[0])
print("decoder_data.shape:", decoder_inputs.shape)

decoder_targets = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')

encoder_data.shape: (20000, 5)
encoder_data[0]: [ 0  0  0  0 11]
decoder_data[0]: [   2 4616    0    0    0    0    0    0    0    0    0]
decoder_data.shape: (20000, 11)


Теперь нужно загрузить предобученные embeddings для английского языка.

In [6]:
word2vec = {}
for line in open(f'glove.6B.{EMBEDDING_DIM}d.txt', encoding='utf-8'):
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
print(f'Найдено {len(word2vec)} векторных представлений слов.')

Найдено 400000 векторных представлений слов.


Из полученных эмбеддингов создадим специальную матрицу, где индекс строки будет соответствовать индексу в словаре word2idx_inputs, что позволит эффективно извлекать соответствующий слову вектор на основе его уникального индекса, данного токенизатором.

In [7]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_inputs.items():
    if i < MAX_NUM_WORDS:
        embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      # ненайденные слова в матрице буду иметь нулевой вектор
      embedding_matrix[i] = embedding_vector

Создадим Embedding layer для Encoder'а и преобразуем целевой признак в OHE, так как не получается использовать sparse_categorical_crossentropy.

In [8]:
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=max_len_input,
)

decoder_targets_one_hot = np.zeros(
  (
    len(input_texts),
    max_len_target,
    num_words_output
  ),
  dtype='float32'
)

for i, d in enumerate(decoder_targets):
    for t, word in enumerate(d):
        if word > 0:
            decoder_targets_one_hot[i, t, word] = 1

Теперь создаем модель.

In [9]:
# сначала Encoder
encoder_inputs_placeholder = Input(shape=(max_len_input,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = Bidirectional(LSTM(LATENT_DIM,return_sequences=True))
encoder_outputs = encoder(x)

# теперь Decoder
decoder_inputs_placeholder = Input(shape=(max_len_target,))
decoder_embedding = Embedding(num_words_output,EMBEDDING_DIM)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

Instructions for updating:
Colocations handled automatically by placer.


Реализуем attention

In [10]:
attn_repeat_layer = RepeatVector(max_len_input)
attn_concat_layer = Concatenate(axis=-1)
attn_dense1 = Dense(10,activation='tanh')
attn_dense2 = Dense(1,activation=softmax_over_time)
attn_dot = Dot(axes=1)

def one_step_attention(h, st_1):
    # h = h(1), ..., h(Tx), shape = (Tx, LATENT_DIM * 2)
    # st_1 = s(t-1), shape = (LATENT_DIM_DECODER,)
 
    # копируем s(t-1) Tx раз
    # теперь shape = (Tx, LATENT_DIM_DECODER)
    st_1 = attn_repeat_layer(st_1)

    # Concatenate all h(t)'s with s(t-1)
    # Теперь shape (Tx, LATENT_DIM_DECODER + LATENT_DIM * 2)
    x = attn_concat_layer([h, st_1])

    x = attn_dense1(x)

    alphas = attn_dense2(x)

    # непосредственно вычисляем контекст
    context = attn_dot([alphas, h])

    return context

Теперь надо написать оставшийся Decoder после реализации attention

In [11]:
decoder_lstm = LSTM(LATENT_DIM_DECODER, return_state=True)
decoder_dense = Dense(num_words_output, activation='softmax')

initial_s = Input(shape=(LATENT_DIM_DECODER,), name='s0')
initial_c = Input(shape=(LATENT_DIM_DECODER,), name='c0')
# для teacher forcing: комбинируем предыдущее правильное слово с текущим контекстом
context_last_word_concat_layer = Concatenate(axis=2)

s = initial_s
c = initial_c

outputs = []
for t in range(max_len_target): # Ty раз
    # вычисляем контекст с использованием attention
    context = one_step_attention(encoder_outputs, s)

    # мы не хотим конкатенировать контекст со всей входной последовательностью для teacher forcing
    # для 1 шага в генерировании выходного слова нам нужно взять лишь 1 слово (правильное на предыдущем шаге)
    selector = Lambda(lambda x: x[:, t:t+1])
    xt = selector(decoder_inputs_x)
  
    # комбинируем 
    decoder_lstm_input = context_last_word_concat_layer([context, xt])

    # передаем комбинированные [контекст, последнее слово] в LSTM
    # вместе с [s, c]
    # получаем новые [s, c] и output
    o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[s, c])

    decoder_outputs = decoder_dense(o)
    outputs.append(decoder_outputs)

In [12]:
# 'outputs' это список длиной Ty
# каждый элемент имеет размер (размер батча, словарь языка на который переводим (русский))
# нам надо преобразовать этот список в 1 тензор
# если просто использовать stack, то получим T x N x D
# а нам нужно вот так N x T x D

def stack_and_transpose(x):
    # x это список длиной Ty, каждый элемент batch_size x output_vocab_size тензор
    x = K.stack(x) # теперь Ty x batch_size x output_vocab_size tensor
    x = K.permute_dimensions(x, pattern=(1, 0, 2)) # теперь batch_size x T x output_vocab_size
    return x

# сделаем из данной функции слой, так как так хочет Keras
stacker = Lambda(stack_and_transpose)
outputs = stacker(outputs)

In [24]:
model = Model(
  inputs=[
    encoder_inputs_placeholder,
    decoder_inputs_placeholder,
    initial_s, 
    initial_c,
  ],
  outputs=outputs
)


def custom_loss(y_true, y_pred):
    # both are of shape N x T x K
    mask = K.cast(y_true > 0, dtype='float32')
    out = mask * y_true * K.log(y_pred)
    return -K.sum(out) / K.sum(mask)


def acc(y_true, y_pred):
    # both are of shape N x T x K
    targ = K.argmax(y_true, axis=-1)
    pred = K.argmax(y_pred, axis=-1)
    correct = K.cast(K.equal(targ, pred), dtype='float32')

    # 0 is padding, don't include those
    mask = K.cast(K.greater(targ, 0), dtype='float32')
    n_correct = K.sum(mask * correct)
    n_total = K.sum(mask)
    return n_correct / n_total


model.compile(optimizer='adam', loss=custom_loss, metrics=[acc])
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

# train the model
z = np.zeros((len(encoder_inputs), LATENT_DIM_DECODER)) # initial [s, c]
r = model.fit(
  [encoder_inputs, decoder_inputs, z, z], decoder_targets_one_hot,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=0.2
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 16000 samples, validate on 4000 samples
Epoch 1/30
16000/16000 [==============================] - 247s 15ms/step - loss: 5.3982 - acc: 0.3393 - val_loss: 5.1719 - val_acc: 0.2978
Epoch 2/30
16000/16000 [==============================] - 233s 15ms/step - loss: 4.2391 - acc: 0.3964 - val_loss: 4.7837 - val_acc: 0.3925
Epoch 3/30
16000/16000 [==============================] - 232s 15ms/step - loss: 3.6410 - acc: 0.4594 - val_loss: 4.5203 - val_acc: 0.4380
Epoch 4/30
16000/16000 [==============================] - 246s 15ms/step - loss: 3.0892 - acc: 0.5048 - val_loss: 4.3843 - val_acc: 0.4687
Epoch 5/30
16000/16000 [==============================] - 218s 14ms/step - loss: 2.5971 - acc: 0.5433 - val_loss: 4.2744 - val_acc: 0.4876
Epoch 6/30
16000/16000 [==============================] - 218s 14ms/step - loss: 2.1474 - acc: 0.5851 - val_loss: 4.2102 - val_acc

Теперь нужно отдельно создать модель для предсказаний.

In [17]:
encoder_model = Model(encoder_inputs_placeholder, encoder_outputs)

# посколько encoder является bidirectional, то на каждом шаге будет два hidden state
encoder_outputs_as_input = Input(shape=(max_len_input, LATENT_DIM * 2,))
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

# нет нужды в цикле, так как тут только 1 шаг
context = one_step_attention(encoder_outputs_as_input, initial_s)
decoder_lstm_input = context_last_word_concat_layer([context, decoder_inputs_single_x])

o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
decoder_outputs = decoder_dense(o)

decoder_model = Model(
  inputs=[
    decoder_inputs_single,
    encoder_outputs_as_input,
    initial_s, 
    initial_c
  ],
  outputs=[decoder_outputs, s, c]
)

# так как модель будет выдавать не слова, а их индексы, то нужно будет их преобразовывать в слова
idx2word_eng = {v:k for k, v in word2idx_inputs.items()}
idx2word_trans = {v:k for k, v in word2idx_outputs.items()}

In [ ]:
def decode_sequence(input_seq):
    # кодируем входную последовательность
    enc_out = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
  
    # начинается перевод со специального токена
    target_seq[0, 0] = word2idx_outputs['<sos>']

    # выходим из цикла, когда встречаем этот токен
    eos = word2idx_outputs['<eos>']


    # [s, c] будут обновляться на каждой итерации
    s = np.zeros((1, LATENT_DIM_DECODER))
    c = np.zeros((1, LATENT_DIM_DECODER))


    # Перевод
    output_sentence = []
    for _ in range(max_len_target):
        o, s, c = decoder_model.predict([target_seq, enc_out, s, c])
        

        # получаем следующее слово
        idx = np.argmax(o.flatten())

        # проверяем конец ли последовательности
        if eos == idx:
            break

        word = ''
        if idx > 0:
            word = idx2word_trans[idx]
            output_sentence.append(word)

        # обновляем входное слово в декодер на следующей итерации
        target_seq[0, 0] = idx

    return ' '.join(output_sentence)

Проверим модель.

In [ ]:
while True:
    # проверка
    i = np.random.choice(len(input_texts))
    input_seq = encoder_inputs[i:i+1]
    translation = decode_sequence(input_seq)
    print('-')
    print('Входное предложение:', input_texts[i])
    print('Предсказанный перевод:', translation)
    print('Истинный перевод:', target_texts[i])

    ans = input("Продолжить? [Y/n]")
    if ans and ans.lower().startswith('n'):
        break